In [21]:
import pandas as pd
import numpy as np
from haversine import haversine, Unit

In [22]:
df = pd.read_csv('data/aisdk-2024-09-11.csv')

In [23]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880299 entries, 0 to 17880298
Data columns (total 26 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   # Timestamp                     object 
 1   Type of mobile                  object 
 2   MMSI                            int64  
 3   Latitude                        float64
 4   Longitude                       float64
 5   Navigational status             object 
 6   ROT                             float64
 7   SOG                             float64
 8   COG                             float64
 9   Heading                         float64
 10  IMO                             object 
 11  Callsign                        object 
 12  Name                            object 
 13  Ship type                       object 
 14  Cargo type                      object 
 15  Width                           float64
 16  Length                          float64
 17  Type of position fixing d

In [18]:
df.iloc[1]

# Timestamp                          11/09/2024 00:00:00
Type of mobile                                   Class A
MMSI                                           220276000
Latitude                                       56.953808
Longitude                                       8.694445
Navigational status               Under way using engine
ROT                                                  0.0
SOG                                                  0.0
COG                                                  NaN
Heading                                             86.0
IMO                                              Unknown
Callsign                                         Unknown
Name                                                 NaN
Ship type                                      Undefined
Cargo type                                           NaN
Width                                                NaN
Length                                               NaN
Type of position fixing device 

In [13]:
def detect_gps_spoofing(df: pd.DataFrame):
    anomalies = []
    for i in range(1, len(df)):
        prev = df.iloc[i - 1]
        curr = df.iloc[i]
        prev_coords = (prev['Latitude'], prev['Longitude'])
        curr_coords = (curr['Latitude'], curr['Longitude'])

        # In km
        distance = haversine(prev_coords, curr_coords, unit=Unit.KILOMETERS)

        # In hours
        time = (pd.to_datetime(
            curr['# Timestamp']) - pd.to_datetime(prev['# Timestamp'])).total_seconds() / 3600.0

        # Calculate speed
        speed = distance / time if time > 0 else 0

        # https://sahyogfreight.com/blog/cargo-ship-speed/#:~:text=The%20optimal%20speed%20of%20a,travel%20at%20around%2015%20knots.
        # 24 knots maximum speed = 44.4kmh
        if speed > 44.4:
            anomalies.append(curr)
    return anomalies

In [9]:
len(detect_gps_spoofing(mmsi_df[0]))

58

In [12]:
mmsi_df[0].size

221728